In [1]:
import os
import json
from typing import Dict, List, Optional, Union, cast
import requests

from env import github_token, github_username

import acquire_zach as az
import wrangle as w

import numpy as np
import pandas as pd

# acquire
from requests import get
from bs4 import BeautifulSoup
from time import sleep
import os

# prepare
import unicodedata
import re
import json
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords

# explore
from sklearn.model_selection import train_test_split
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

# model
from pprint import pprint
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report

# Use helper functions to acquire a list of url suffixes from all of Google's Github Repos

In [2]:
def get_githubpgs():
    """
    This function will create a list of all the url pages in the google repos section
    """
    urls = []
    for i in range(1,68):
        
        url = f'https://github.com/google?page={i}'
        urls.append(url)
    return urls

In [3]:
url = 'https://github.com/google?page=1'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
response = requests.get(url, headers=headers)

html = response.text
soup = BeautifulSoup(html)

In [4]:
def get_repo_linksuffix():
    """
    This function will concat a base url with the repo suffix
    and create a list of all appended repo links on a single page.
    """
    
    repos = []
    for i in range(0,29):
        link_suffix = soup.find_all('a', itemprop='name codeRepository')[i].get('href')
        repos.append(link_suffix)
    return repos


In [5]:
def souper(site):
    """
    
    """
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    response = get(site, headers=headers)
    
    html = response.text
    soup = BeautifulSoup(html)
    
    return soup

In [6]:
def get_all_repo_links():
    """
    
    """
    repo_links = []
    
    urls = get_githubpgs()
    
    for url in urls:
        soup = souper(url)
        
        repos = get_repo_linksuffix()
        repo_links.append(repos)
    
    return repo_links

In [8]:
# run through all helper functions to create a nested list of the repo urls
repo_links = get_all_repo_links()

In [9]:
len(repo_links)

67

In [10]:
repo_links

[['/google/BIG-bench',
  '/google/megalista',
  '/google/timesketch',
  '/google/iree',
  '/google/highway',
  '/google/agi',
  '/google/oss-fuzz',
  '/google/llvm-bazel',
  '/google/truth',
  '/google/clspv',
  '/google/site-kit-wp',
  '/google/nomulus',
  '/google/conscrypt',
  '/google/ground-platform',
  '/google/it-cert-automation-practice',
  '/google/ExoPlayer',
  '/google/zx',
  '/google/typedoc-neo-theme',
  '/google/go-github',
  '/google/perfetto',
  '/google/flax',
  '/google/orbit',
  '/google/shaka-packager',
  '/google/rrg',
  '/google/benchmark',
  '/google/mdl-ref-apps',
  '/google/zazu',
  '/google/accompanist',
  '/google/web-stories-wp'],
 ['/google/BIG-bench',
  '/google/megalista',
  '/google/timesketch',
  '/google/iree',
  '/google/highway',
  '/google/agi',
  '/google/oss-fuzz',
  '/google/llvm-bazel',
  '/google/truth',
  '/google/clspv',
  '/google/site-kit-wp',
  '/google/nomulus',
  '/google/conscrypt',
  '/google/ground-platform',
  '/google/it-cert-automa

In [11]:
# flatten the nested list into a single list
import itertools
flatten_list = list(itertools.chain(*repo_links))

In [12]:
len(flatten_list)

1943

In [13]:
flatten_list

['/google/BIG-bench',
 '/google/megalista',
 '/google/timesketch',
 '/google/iree',
 '/google/highway',
 '/google/agi',
 '/google/oss-fuzz',
 '/google/llvm-bazel',
 '/google/truth',
 '/google/clspv',
 '/google/site-kit-wp',
 '/google/nomulus',
 '/google/conscrypt',
 '/google/ground-platform',
 '/google/it-cert-automation-practice',
 '/google/ExoPlayer',
 '/google/zx',
 '/google/typedoc-neo-theme',
 '/google/go-github',
 '/google/perfetto',
 '/google/flax',
 '/google/orbit',
 '/google/shaka-packager',
 '/google/rrg',
 '/google/benchmark',
 '/google/mdl-ref-apps',
 '/google/zazu',
 '/google/accompanist',
 '/google/web-stories-wp',
 '/google/BIG-bench',
 '/google/megalista',
 '/google/timesketch',
 '/google/iree',
 '/google/highway',
 '/google/agi',
 '/google/oss-fuzz',
 '/google/llvm-bazel',
 '/google/truth',
 '/google/clspv',
 '/google/site-kit-wp',
 '/google/nomulus',
 '/google/conscrypt',
 '/google/ground-platform',
 '/google/it-cert-automation-practice',
 '/google/ExoPlayer',
 '/goog

In [18]:
# remove the first unnecessary '/' character 
# using Pandas Series
reposuffixes = pd.Series(flatten_list).str[1:]

In [22]:
# convert Series back to a list to feed into acquire.py script 
# to pull a .json file of all READMEs

REPOS = reposuffixes.to_list()

['google/BIG-bench',
 'google/megalista',
 'google/timesketch',
 'google/iree',
 'google/highway',
 'google/agi',
 'google/oss-fuzz',
 'google/llvm-bazel',
 'google/truth',
 'google/clspv',
 'google/site-kit-wp',
 'google/nomulus',
 'google/conscrypt',
 'google/ground-platform',
 'google/it-cert-automation-practice',
 'google/ExoPlayer',
 'google/zx',
 'google/typedoc-neo-theme',
 'google/go-github',
 'google/perfetto',
 'google/flax',
 'google/orbit',
 'google/shaka-packager',
 'google/rrg',
 'google/benchmark',
 'google/mdl-ref-apps',
 'google/zazu',
 'google/accompanist',
 'google/web-stories-wp',
 'google/BIG-bench',
 'google/megalista',
 'google/timesketch',
 'google/iree',
 'google/highway',
 'google/agi',
 'google/oss-fuzz',
 'google/llvm-bazel',
 'google/truth',
 'google/clspv',
 'google/site-kit-wp',
 'google/nomulus',
 'google/conscrypt',
 'google/ground-platform',
 'google/it-cert-automation-practice',
 'google/ExoPlayer',
 'google/zx',
 'google/typedoc-neo-theme',
 'google/

In [25]:
import os
import json
from typing import Dict, List, Optional, Union, cast
import requests

from env import github_token, github_username


headers = {"Authorization": f"token {github_token}", "User-Agent": github_username}

if headers["Authorization"] == "token " or headers["User-Agent"] == "":
    raise Exception(
        "You need to follow the instructions marked TODO in this script before trying to use it"
    )


def github_api_request(url: str) -> Union[List, Dict]:
    response = requests.get(url, headers=headers)
    response_data = response.json()
    if response.status_code != 200:
        raise Exception(
            f"Error response from github api! status code: {response.status_code}, "
            f"response: {json.dumps(response_data)}"
        )
    return response_data


def get_repo_language(repo: str) -> str:
    url = f"https://api.github.com/repos/{repo}"
    repo_info = github_api_request(url)
    if type(repo_info) is dict:
        repo_info = cast(Dict, repo_info)
        if "language" not in repo_info:
            raise Exception(
                "'language' key not round in response\n{}".format(json.dumps(repo_info))
            )
        return repo_info["language"]
    raise Exception(
        f"Expecting a dictionary response from {url}, instead got {json.dumps(repo_info)}"
    )


def get_repo_contents(repo: str) -> List[Dict[str, str]]:
    url = f"https://api.github.com/repos/{repo}/contents/"
    contents = github_api_request(url)
    if type(contents) is list:
        contents = cast(List, contents)
        return contents
    raise Exception(
        f"Expecting a list response from {url}, instead got {json.dumps(contents)}"
    )


def get_readme_download_url(files: List[Dict[str, str]]) -> str:
    """
    Takes in a response from the github api that lists the files in a repo and
    returns the url that can be used to download the repo's README file.
    """
    for file in files:
        if file["name"].lower().startswith("readme"):
            return file["download_url"]
    return ""


def process_repo(repo: str) -> Dict[str, str]:
    """
    Takes a repo name like "gocodeup/codeup-setup-script" and returns a
    dictionary with the language of the repo and the readme contents.
    """
    contents = get_repo_contents(repo)
    readme_download_url = get_readme_download_url(contents)
    if readme_download_url == "":
        readme_contents = ""
    else:
        readme_contents = requests.get(readme_download_url).text
    return {
        "repo": repo,
        "language": get_repo_language(repo),
        "readme_contents": readme_contents,
    }


def scrape_github_data() -> List[Dict[str, str]]:
    """
    Loop through all of the repos and process them. Returns the processed data.
    """
    return [process_repo(repo) for repo in REPOS]


if __name__ == "__main__":
    data = scrape_github_data()
    json.dump(data, open("data.json", "w"), indent=1)

In [26]:
# run acquire.py script to create a .json file of all READMEs
# this takes awhile and produced an error message
# that I used my token too much
# but still produced complete .json file
scrape_github_data()

Exception: Error response from github api! status code: 403, response: {"message": "API rate limit exceeded for user ID 48535351.", "documentation_url": "https://docs.github.com/rest/overview/resources-in-the-rest-api#rate-limiting"}

In [27]:
f = open('data.json')

In [28]:
data = json.load(f)

In [29]:
df = pd.DataFrame(data)

In [30]:
# create DF of all READMEs
df.head(2)

,repo,language,readme_contents
0,google/BIG-bench,Jupyter Notebook,# BIG-bench :chair:\n\nThe Beyond the Imitatio...
1,google/megalista,Python,# Megalista\n\nSample integration code for onb...


In [36]:
df.to_csv('google_readmes.csv')

In [ ]:
df = read_csv('')

In [34]:
import wrangle as w

In [35]:
# transform into a series

string = " ".join(df.readme_contents)

string = " ".join(w.clean(string))

NameError: name 'nltk' is not defined